In [1]:
import torch
from torchvision import transforms, models
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
from models.shashimal2 import Shashimal2
from models.__init__ import save_checkpoint, resume_checkpoint
#from dataloader.shashimal2_synth import GooDataset
from dataloader.shashimal2 import GooDataset
from dataloader import chong_imutils
from training.train_shashimal2 import train, test, GazeOptimizer, test_gop, train_with_early_stopping
from torch.nn.utils.rnn import pad_sequence

In [3]:
batch_size=4
workers=4

images_dir = '/media/primesh/F4D0EA80D0EA4906/PROJECTS/FYP/Gaze detection/Datasets/gooreal/finalrealdatasetImgsV2'
pickle_path = '/media/primesh/F4D0EA80D0EA4906/PROJECTS/FYP/Gaze detection/Datasets/gooreal/oneshotrealhumansNew.pickle'
test_images_dir = '/media/primesh/F4D0EA80D0EA4906/PROJECTS/FYP/Gaze detection/Datasets/gooreal/finalrealdatasetImgsV2'
test_pickle_path = '/media/primesh/F4D0EA80D0EA4906/PROJECTS/FYP/Gaze detection/Datasets/gooreal/testrealhumansNew.pickle'

def pad_x_collate_function(batch):
    # batch looks like [(x0,y0), (x4,y4), (x2,y2)... ]
    img, face, head_channel, object_channel,gaze_final,eye,gaze_idx,gt_bboxes,gt_labels = zip(*batch)

    # If you want to be a little fancy, you can do the above in one line
    # xs, ys = zip(*samples)
    img = pad_sequence(img, batch_first=True, padding_value=0)
    face = pad_sequence(face, batch_first=True, padding_value=0)
    head_channel = pad_sequence(head_channel, batch_first=True, padding_value=0)
    object_channel = pad_sequence(object_channel, batch_first=True, padding_value=0)
    #eye = pad_sequence(eye, batch_first=True, padding_value=0)
    #gaze = pad_sequence(gaze, batch_first=True, padding_value=0)
    #gtbox = pad_sequence(gtbox, batch_first=True, padding_value=0)
    #gt_bboxes = pad_sequence(gt_bboxes, batch_first=True, padding_value=0)
    return img, face, head_channel, object_channel,gaze_final,eye,gaze_idx,zip(*gt_bboxes),zip(*gt_labels)

train_set = GooDataset(images_dir, pickle_path, 'train')
train_data_loader = DataLoader(dataset=train_set,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=4)

test_set = GooDataset(test_images_dir, test_pickle_path, 'test')
test_data_loader = DataLoader(test_set, batch_size=1,
                            shuffle=False, num_workers=4, collate_fn=pad_x_collate_function)

Number of Images: 2450
Number of Images: 2146


In [4]:
img, face, head_channel, object_channel, gaze_final, eye, gaze_idx, gt_bboxes, gt_labels  = next(iter(test_data_loader))

In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [15]:
image = img.cuda()
head_channel = head_channel.cuda()
face = face.cuda()
object_channel = object_channel.cuda()
gt_bboxes = np.array(list(gt_bboxes))
gt_labels = np.array(list(gt_labels))
gaze_idx = np.array(gaze_idx)
# outputs, raw_hm = model.raw_hm(image, face, head_channel, object_channel)

In [31]:
image.shape

torch.Size([1, 3, 224, 224])

In [16]:
gt_bboxes.shape

(84, 1, 4)

In [17]:
b0=gt_bboxes[0]
b0=(b0 * 224).astype(int)
b0

array([[11, 63, 22, 97]])

In [18]:
image.shape

torch.Size([1, 3, 224, 224])

In [25]:
i=image.squeeze().detach().cpu().numpy().transpose(1,2,0)

In [20]:
b=np.zeros((224,224), dtype='float32')

In [21]:
bbox_l=[]
for i in range(gt_bboxes.shape[0]):
    bbox=(gt_bboxes[i]*224).astype(int)
    xmin, ymin, xmax, ymax = bbox[0]
    b=np.zeros((224,224), dtype='float32')
    assert xmin<xmax
    assert ymin<ymax
    for j in range(ymin, ymax):
        for k in range(xmin, xmax):
            b[j][k]=1
    bbox_l.append(b)

In [26]:
i.shape

(224, 224, 3)

In [27]:
plt.imshow(i)
plt.imshow(bbox_l[13], 'jet', interpolation='none', alpha=0.8)

plt.savefig('bbox.png')

In [23]:
plt.imshow(b)
plt.show()

<ipython-input-23-417241893c3c>:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [28]:
import gc

gc.collect()

torch.cuda.empty_cache()


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = Shashimal2()


model_ft = model_ft.to(device)

criterion = nn.NLLLoss().cuda()
# criterion =

# Observe that all parameters are being optimized
start_epoch = 0
max_epoch = 5
learning_rate = 1e-4

# Initializes Optimizer
gaze_opt = GazeOptimizer(model_ft, learning_rate)
optimizer = gaze_opt.getOptimizer(start_epoch)
if True:
    checkpoint_fpath = '/media/primesh/F4D0EA80D0EA4906/PROJECTS/FYP/Gaze detection/code/ObjectGazeNet/saved_weights/shashimal2_gazefollow_6_gooreal_16_chechkpoint_full.pt'
    checkpoint = torch.load(checkpoint_fpath)
    model_ft.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])


26


In [35]:
object_channel.shape

torch.Size([1, 1, 224, 224])

In [38]:
image = img.cuda()
head_channel = head_channel.cuda()
face = face.cuda()
object_channel = object_channel.cuda()
outputs, raw_hm = model_ft.raw_hm(image, face, head_channel, object_channel)

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 32, 1, 1])